# Laboratorio 4, Color Iris

Utilizando la base de datos de flores y el método de KNN categorizar cada una de ellas para
encontrar el valor óptimo de K siguiendo los siguientes parámetros o requerimientos de
diseño:

    • NO SE PERMITE UTILIZAR LIBRERIAS DE SCKLEARN, TODO EL PROCEDIMIENTO TIENE
    QUE SER ESCRITO DESDE CERO
    • La cantidad de datos utilizados por cada categoría para evaluarlos debe ser el 20 %
    de esta. La cual debe ser seleccionada de forma aleatoria. El restante 80 % tiene que
    ser utilizado para estimar el mejor valor de K
    • Deben aplicar la distancia euclidiana con mínimo 2 variables. NO SE ACEPTAN
    SOLAMENTE 1.
    • Deben almacenar los vectores de diferencia por cada K probado.
    • Deben reportar por cada K probado la cantidad de aciertos y fallas realizadas por el
    algoritmo. Seleccione el K que mejor desempeño obtuvo mirando solamente la
    cantidad de aciertos y fallos.
    • Deben probar con mínimo 5 valores de K. iniciando desde K=3. NO SE ACEPTA UN K
    INFERIOR A 3
    • Con el mejor K seleccionado, ahora deben seleccionar el 50% de cada categoría y
    probar el desempeño en la clasificación.
    • Con el mejor K seleccionado, ahora deben seleccionar el 75% de cada categoría y
    probar el desempeño en la clasificación.
    • Analicen los resultados obtenidos y den una razón el por qué se podría obtener
    desempeños diferentes al probarlos con diferentes cantidades de datos. 


A continuación se importa el dataset y se le asignan sus respectivos nombres a las columnas

In [1]:
# paquetes de impotacion y manejo de datos
import pandas as pd
import numpy as np

import matplotlib.pyplot as plot

archivo = 'color_iris.csv'

ds = pd.read_csv(archivo)

nombre_columnas = ['sepal_length', 'sepal_width',
                   'petal_length', 'petal_width', 'class']

ds.columns = nombre_columnas

ds


,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


Aqui se definen los porcentajes de datos a ser usados los cuales son el 20% de datos para la primera prueba, y luego el 50% y 70% para probar con el mejor K hallado con los datos del 20% hallados inicialmente

In [24]:
import math

veinte_porciento = math.ceil(149*0.2)  # elegir el 20% de datos
print(veinte_porciento)

cincuenta_porciento =  math.ceil(149*0.5)
print(cincuenta_porciento)

setenta_cinco_porciento = math.ceil(149*0.75)
print(setenta_cinco_porciento)

30
75
112


Aquí se revuelven los datos del dataset para que queden de manera aleatoria

In [7]:
df_revuelto = ds.sample(frac=1)

df_revuelto


,sepal_length,sepal_width,petal_length,petal_width,class
17,5.7,3.8,1.7,0.3,Iris-setosa
50,6.4,3.2,4.5,1.5,Iris-versicolor
53,6.5,2.8,4.6,1.5,Iris-versicolor
40,4.5,2.3,1.3,0.3,Iris-setosa
42,5.0,3.5,1.6,0.6,Iris-setosa
...,...,...,...,...,...
100,5.8,2.7,5.1,1.9,Iris-virginica
133,6.1,2.6,5.6,1.4,Iris-virginica
62,6.1,2.9,4.7,1.4,Iris-versicolor
13,5.8,4.0,1.2,0.2,Iris-setosa


Se crea la funcion que nos permite hallar la distancia euclidiana, esto es para poder saber la distancia entre los datos del 20% y el 80% en el primer caso, de esta manera se hallan mediante la hipotenusa, donde para hallar esta se hallan primero los catetos mediante sus cordenadas.

In [8]:
def distancia_euclidiana(df_revuelto, porcentaje_test):

    distancias = []
    for i in range(0, porcentaje_test):
        distancia = []
        for j in range(porcentaje_test, 149):
            y_prueba = df_revuelto[['sepal_length', 'sepal_width']].iloc[i][0]
            x_prueba = df_revuelto[['sepal_length', 'sepal_width']].iloc[i][1]

            y_medir = df_revuelto[['sepal_length', 'sepal_width']].iloc[j][0]
            x_medir = df_revuelto[['sepal_length', 'sepal_width']].iloc[j][1]

            cateto1 = abs(y_prueba - y_medir) #Hallar el valor de los catetos del triangulo
            cateto2 = abs(x_prueba - x_medir)

            hipotenusa = math.sqrt((cateto1**2)+(cateto2**2)) #Con los catetos ya hallados, podemos saber el valor de la hipotenusa la cual sabemos representa la distancia de un punto a otro

            distancia.append(
                (hipotenusa, df_revuelto.index[j], df_revuelto.index[i])) #Se guarda la distancia de un punto a otro, el indice de los valores medidos y el indice de los valores de test (20%)

        distancias.append(distancia)

    return distancias


Ahora aplicamos la función para el 20% de los datos aleatorios, con esta funcion almacenamos los indices de los valores a medir y guardamos tambien el index de los valores de test o las clases reales. Esto nos servira para luego comparar si el mejor K funciona para determinar la clase sin conocer cual es su clase

In [22]:
distancias = distancia_euclidiana(df_revuelto, veinte_porciento)
distancias

[[(1.004987562112089, 113, 17),
  (1.0816653826391966, 102, 17),
  (0.6000000000000005, 14, 17),
  (0.31622776601683783, 15, 17),
  (1.1313708498984758, 0, 17),
  (0.9055385138137416, 63, 17),
  (1.1999999999999997, 78, 17),
  (2.0, 116, 17),
  (0.31622776601683766, 9, 17),
  (1.0630145812734646, 8, 17),
  (1.1401754250991376, 82, 17),
  (1.2041594578792292, 91, 17),
  (0.9486832980505137, 77, 17),
  (0.8999999999999999, 95, 17),
  (0.49999999999999983, 30, 17),
  (1.1313708498984758, 146, 17),
  (1.2041594578792292, 74, 17),
  (1.118033988749895, 123, 17),
  (1.0630145812734646, 33, 17),
  (1.118033988749895, 125, 17),
  (1.22065556157337, 64, 17),
  (1.6401219466856725, 107, 17),
  (1.22065556157337, 139, 17),
  (1.5264337522473745, 37, 17),
  (1.6552945357246849, 92, 17),
  (0.6000000000000005, 43, 17),
  (1.6124515496597092, 101, 17),
  (1.3416407864998738, 119, 17),
  (0.7810249675906659, 22, 17),
  (2.2, 130, 17),
  (0.8544003745317529, 137, 17),
  (1.6763054614240207, 67, 17),
 

Primero se determinan los valores de K en una lista

Ahora se crea la funcion para hallar K:

1. Se ordenan los datos de menor a mayor
2. Ahora solo se guardan los valores por la cantidad de K ingresada, por ejemplo si nuestr K es 3, debería usar solo los 3 primeros menores de los datos
3. Se guardan las clases de estos valores con menor distancia
4. Ahora se saca la clase mas común entre los datos seleccionados según el K y por su cercania
5. Se compara luego si la clase elegida mediante los K es igual a la clase original del dataset
6. Luego se almacenan el numero de aciertos 
7. Y por ultimo se retorna el "porcentaje" de aciertos según el total de distancias


In [23]:
from collections import Counter

##El numero comentado sera el numero del paso mencionado anteriormente

k = [3, 11, 5, 9, 17] 


def hallarK(distancias, k):
    aciertos = 0
    for i in range(0, len(distancias)):
        distancias_sort = sorted(
            distancias[i], key=lambda distancia: distancia[0]) #1
        sub_list = []
        for flor in distancias_sort[0:k]: #2
            nombre_flor = ds.at[flor[1], 'class']
            sub_list.append(nombre_flor) #3

        contador = Counter(sub_list)
        elemento_mas_comun = contador.most_common(1)[0][0] #4
        
        nombre_flor_real = ds.at[distancias_sort[0:k][0][2], 'class'] 
        if (str(elemento_mas_comun) == nombre_flor_real): #5
            aciertos += 1 #6
    return aciertos/len(distancias) #7


Aquí se usa la funcion para lograr probar todos los valores de K y poder despues determinar el mejor K, en este caso el mejor K es 17 el cual nos da un 73,3% de aciertos

In [28]:
for i in k:
    print('K = ', i)
    print(hallarK(distancias, i))


K =  3
0.7
K =  11
0.7
K =  5
0.6666666666666666
K =  9
0.6666666666666666
K =  17
0.7333333333333333


Una vez sacado el mejor K, se procede a probar de nuevo la distancia euclidiana con el 50% de los datos

In [18]:
distancias = distancia_euclidiana(df_revuelto, cincuenta_porciento)
distancias

[[(1.61245154965971, 12, 17),
  (1.1180339887498953, 21, 17),
  (1.2165525060596436, 89, 17),
  (2.33238075793812, 117, 17),
  (0.7999999999999998, 94, 17),
  (0.9219544457292886, 114, 17),
  (1.3892443989449805, 51, 17),
  (0.49999999999999983, 84, 17),
  (0.8544003745317529, 83, 17),
  (0.4472135954999583, 32, 17),
  (1.2529964086141672, 46, 17),
  (1.2206555615733703, 131, 17),
  (1.9313207915827966, 59, 17),
  (1.1045361017187258, 81, 17),
  (1.004987562112089, 120, 17),
  (0.8062257748298548, 65, 17),
  (0.58309518948453, 26, 17),
  (0.9999999999999997, 109, 17),
  (1.5264337522473745, 105, 17),
  (1.1045361017187258, 93, 17),
  (2.1540659228538015, 134, 17),
  (0.9848857801796107, 10, 17),
  (1.414213562373095, 79, 17),
  (0.6403124237432848, 147, 17),
  (2.06155281280883, 104, 17),
  (1.118033988749895, 143, 17),
  (0.49999999999999983, 19, 17),
  (0.6000000000000005, 45, 17),
  (1.170469991071963, 5, 17),
  (1.3892443989449805, 140, 17),
  (0.7810249675906652, 99, 17),
  (1.131

Nuestro K con el 50% de los datos nos da 78,6%

In [19]:
print(hallarK(distancias, 17))

0.7866666666666666


Una vez sacado el mejor K el cual es 17, se procede a probar de nuevo la distancia euclidiana con el 75% de los datos

In [29]:
distancias = distancia_euclidiana(df_revuelto, setenta_cinco_porciento)
distancias

[[(0.7280109889280519, 3, 17),
  (1.3038404810405295, 68, 17),
  (0.8602325267042628, 48, 17),
  (0.4123105625617663, 47, 17),
  (1.0630145812734648, 24, 17),
  (0.31622776601683783, 4, 17),
  (1.0295630140987, 96, 17),
  (1.140175425099138, 29, 17),
  (1.4317821063276355, 97, 17),
  (0.8944271909999155, 126, 17),
  (0.761577310586391, 39, 17),
  (1.5811388300841893, 7, 17),
  (1.3892443989449805, 138, 17),
  (1.0630145812734646, 36, 17),
  (1.4866068747318502, 75, 17),
  (1.140175425099138, 73, 17),
  (0.806225774829855, 6, 17),
  (1.2806248474865696, 76, 17),
  (0.9219544457292886, 34, 17),
  (1.0770329614269007, 72, 17),
  (1.272792206135785, 57, 17),
  (1.431782106327635, 49, 17),
  (0.9899494936611665, 136, 17),
  (1.6124515496597098, 56, 17),
  (0.3605551275463989, 35, 17),
  (0.9848857801796107, 23, 17),
  (1.3152946437965904, 88, 17),
  (1.0, 98, 17),
  (0.7211102550927976, 135, 17),
  (1.6278820596099701, 61, 17),
  (1.2041594578792296, 11, 17),
  (1.9723082923316022, 129, 17)

Nuestro K con el 70% de los datos nos da 59.8%

In [30]:
print(hallarK(distancias, 17))

0.5982142857142857


# Conclusiones

Se puede determinar con el 20% de los datos se ha hallado la distancia euclidiana para encontrar los K vecinos más cercanos para cada objeto en el conjunto de prueba, y se ha encontrado que el valor óptimo de K es 17, que da una tasa de aciertos del 73.3%.

Después, se ha utilizado el mismo valor de K (17) para clasificar otro conjunto de datos, pero con diferentes proporciones de medicion y prueba. Cuando se ha utilizado el 50% de los datos para probar el modelo, se ha obtenido una tasa de aciertos del 78.6% y se logra observar que el modelo tiene una tasa de aciertos mayor del 78%. Esto puede deberse a que los datos de prueba utilizados son diferentes y, por lo tanto, el modelo se ajusta mejor a esos datos.

Sin embargo, cuando se ha utilizado el 75% de los datos para probar el modelo, la tasa de aciertos ha disminuido significativamente al 59.8%. Esto puede deberse a que la cantidad de datos de entrenamiento se ha reducido significativamente en comparación con la cantidad de datos de prueba, esto puede deberse a que el conjunto de datos de prueba utilizado es demasiado grande y no representa adecuadamente la distribución de los datos de medicion, lo que hace que el modelo no pueda generalizar bien en el conjunto de prueba.

En resumen, el rendimiento del modelo KNN puede variar según el conjunto de datos de medicion y de prueba utilizados. Es importante ajustar el valor de k adecuadamente y utilizar un conjunto de prueba representativo para obtener una tasa de aciertos óptima en el modelo.